# Selected Economic Characteristics: Occupation from the American Community Survey

**[Work in progress]**

This notebook downloads [selected economic characteristics (DP03)](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03) from the American Community Survey 2018 5-Year Data.

Data source: [American Community Survey 5-Year Data](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [1]:
import os
import pandas as pd
from pathlib import Path
import time

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-19636412-9e74-4bac-8a4c-c6c8b49bb9d3/installation-4.1.0/import


## Download selected variables

* [Selected economic characteristics for US](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03/)

* [Description of variables](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2018_ACSSubjectDefinitions.pdf)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP03 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

In [4]:
variables = {# OCCUPATION
             'DP03_0026E': 'civilianEmployedPopulation16YearsAndOver',
             'DP03_0027E': 'managementBusinessScienceAndArtsOccupations',
             'DP03_0027PE': 'managementBusinessScienceAndArtsOccupationsPct',
             'DP03_0028E': 'serviceOccupations',
             'DP03_0028PE': 'serviceOccupationsPct',
             'DP03_0029E': 'salesAndOfficeOccupations',
             'DP03_0029PE': 'salesAndOfficeOccupationsPct',
             'DP03_0030E': 'naturalResourcesConstructionAndMaintenanceOccupations',
             'DP03_0030PE': 'naturalResourcesConstructionAndMaintenanceOccupationsPct',
             'DP03_0031E': 'productionTransportationAndMaterialMovingOccupations',
             'DP03_0031PE': 'productionTransportationAndMaterialMovingOccupationsPct'
            }

In [5]:
fields = ",".join(variables.keys())

In [6]:
for v in variables.values():
    if 'Pct' in v:
        print('o.' + v + ' = toFloat(row.' + v + '),')
    else:
        print('o.' + v + ' = toInteger(row.' + v + '),')

o.civilianEmployedPopulation16YearsAndOver = toInteger(row.civilianEmployedPopulation16YearsAndOver),
o.managementBusinessScienceAndArtsOccupations = toInteger(row.managementBusinessScienceAndArtsOccupations),
o.managementBusinessScienceAndArtsOccupationsPct = toFloat(row.managementBusinessScienceAndArtsOccupationsPct),
o.serviceOccupations = toInteger(row.serviceOccupations),
o.serviceOccupationsPct = toFloat(row.serviceOccupationsPct),
o.salesAndOfficeOccupations = toInteger(row.salesAndOfficeOccupations),
o.salesAndOfficeOccupationsPct = toFloat(row.salesAndOfficeOccupationsPct),
o.naturalResourcesConstructionAndMaintenanceOccupations = toInteger(row.naturalResourcesConstructionAndMaintenanceOccupations),
o.naturalResourcesConstructionAndMaintenanceOccupationsPct = toFloat(row.naturalResourcesConstructionAndMaintenanceOccupationsPct),
o.productionTransportationAndMaterialMovingOccupations = toInteger(row.productionTransportationAndMaterialMovingOccupations),
o.productionTransportati

In [7]:
print(len(variables.keys()))

11


## Download county-level data using US Census API

In [8]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [9]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,DP03_0026E,DP03_0027E,DP03_0027PE,DP03_0028E,DP03_0028PE,DP03_0029E,DP03_0029PE,DP03_0030E,DP03_0030PE,DP03_0031E,DP03_0031PE,state,county
1,17133,5150,30.1,3193,18.6,4179,24.4,1540,9.0,3071,17.9,28,151
2,4591,1190,25.9,917,20.0,814,17.7,651,14.2,1019,22.2,28,111
3,3097,800,25.8,522,16.9,640,20.7,497,16.0,638,20.6,28,019
4,9509,2316,24.4,1688,17.8,1920,20.2,1104,11.6,2481,26.1,28,057


##### Add column names

In [10]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [11]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [12]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [13]:
df.head()

,civilianEmployedPopulation16YearsAndOver,managementBusinessScienceAndArtsOccupations,managementBusinessScienceAndArtsOccupationsPct,serviceOccupations,serviceOccupationsPct,salesAndOfficeOccupations,salesAndOfficeOccupationsPct,naturalResourcesConstructionAndMaintenanceOccupations,naturalResourcesConstructionAndMaintenanceOccupationsPct,productionTransportationAndMaterialMovingOccupations,productionTransportationAndMaterialMovingOccupationsPct,stateFips,countyFips
1,17133,5150,30.1,3193,18.6,4179,24.4,1540,9.0,3071,17.9,28,151
2,4591,1190,25.9,917,20.0,814,17.7,651,14.2,1019,22.2,28,111
3,3097,800,25.8,522,16.9,640,20.7,497,16.0,638,20.6,28,019
4,9509,2316,24.4,1688,17.8,1920,20.2,1104,11.6,2481,26.1,28,057
5,3466,1253,36.2,539,15.6,499,14.4,596,17.2,579,16.7,28,015


In [14]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,civilianEmployedPopulation16YearsAndOver,managementBusinessScienceAndArtsOccupations,managementBusinessScienceAndArtsOccupationsPct,serviceOccupations,serviceOccupationsPct,salesAndOfficeOccupations,salesAndOfficeOccupationsPct,naturalResourcesConstructionAndMaintenanceOccupations,naturalResourcesConstructionAndMaintenanceOccupationsPct,productionTransportationAndMaterialMovingOccupations,productionTransportationAndMaterialMovingOccupationsPct,stateFips,countyFips
1869,1564930,652475,41.7,304726,19.5,340038,21.7,119478,7.6,148213,9.5,06,073


In [15]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [16]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03OccupationAdmin2.csv", index=False)

## Download zip-level data using US Census API

In [17]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [18]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,DP03_0026E,DP03_0027E,DP03_0027PE,DP03_0028E,DP03_0028PE,DP03_0029E,DP03_0029PE,DP03_0030E,DP03_0030PE,DP03_0031E,DP03_0031PE,zip code tabulation area
1,3564,989,27.7,769,21.6,786,22.1,263,7.4,757,21.2,43964
2,25800,9444,36.6,4847,18.8,5727,22.2,1783,6.9,3999,15.5,28216
3,37198,21707,58.4,3448,9.3,8901,23.9,1327,3.6,1815,4.9,28277
4,15182,6793,44.7,1814,11.9,4007,26.4,813,5.4,1755,11.6,28278


##### Add column names

In [19]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('postalCode')
df.columns = columns

In [20]:
df.head()

,civilianEmployedPopulation16YearsAndOver,managementBusinessScienceAndArtsOccupations,managementBusinessScienceAndArtsOccupationsPct,serviceOccupations,serviceOccupationsPct,salesAndOfficeOccupations,salesAndOfficeOccupationsPct,naturalResourcesConstructionAndMaintenanceOccupations,naturalResourcesConstructionAndMaintenanceOccupationsPct,productionTransportationAndMaterialMovingOccupations,productionTransportationAndMaterialMovingOccupationsPct,postalCode
1,3564,989,27.7,769,21.6,786,22.1,263,7.4,757,21.2,43964
2,25800,9444,36.6,4847,18.8,5727,22.2,1783,6.9,3999,15.5,28216
3,37198,21707,58.4,3448,9.3,8901,23.9,1327,3.6,1815,4.9,28277
4,15182,6793,44.7,1814,11.9,4007,26.4,813,5.4,1755,11.6,28278
5,11384,3855,33.9,2554,22.4,2989,26.3,690,6.1,1296,11.4,28303


In [21]:
# Example data
df.query("postalCode == '90210'")

,civilianEmployedPopulation16YearsAndOver,managementBusinessScienceAndArtsOccupations,managementBusinessScienceAndArtsOccupationsPct,serviceOccupations,serviceOccupationsPct,salesAndOfficeOccupations,salesAndOfficeOccupationsPct,naturalResourcesConstructionAndMaintenanceOccupations,naturalResourcesConstructionAndMaintenanceOccupationsPct,productionTransportationAndMaterialMovingOccupations,productionTransportationAndMaterialMovingOccupationsPct,postalCode
30897,8687,5773,66.5,676,7.8,1943,22.4,59,0.7,236,2.7,90210


In [22]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [23]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03OccupationZip.csv", index=False)

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [24]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.sleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [25]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [26]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [27]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [28]:
# Example data for San Diego County
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

,civilianEmployedPopulation16YearsAndOver,managementBusinessScienceAndArtsOccupations,managementBusinessScienceAndArtsOccupationsPct,serviceOccupations,serviceOccupationsPct,salesAndOfficeOccupations,salesAndOfficeOccupationsPct,naturalResourcesConstructionAndMaintenanceOccupations,naturalResourcesConstructionAndMaintenanceOccupationsPct,productionTransportationAndMaterialMovingOccupations,productionTransportationAndMaterialMovingOccupationsPct,stateFips,countyFips,tract,source,aggregationLevel
56,3751,2456,65.5,400,10.7,668,17.8,157,4.2,70,1.9,06,073,06073008324,American Community Survey 5 year,Tract
57,1282,718,56.0,197,15.4,260,20.3,23,1.8,84,6.6,06,073,06073008339,American Community Survey 5 year,Tract
58,3720,2017,54.2,678,18.2,563,15.1,189,5.1,273,7.3,06,073,06073008347,American Community Survey 5 year,Tract
59,5535,3160,57.1,761,13.7,1078,19.5,147,2.7,389,7.0,06,073,06073008354,American Community Survey 5 year,Tract
60,3349,1374,41.0,802,23.9,617,18.4,236,7.0,320,9.6,06,073,06073008505,American Community Survey 5 year,Tract


### Save data

In [29]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03OccupationTract.csv", index=False)

In [30]:
df.shape

(73056, 16)